In [1]:
# Make SQLite database
# https://docs.python.org/3/library/sqlite3.html
# https://stackoverflow.com/questions/4654762/how-can-one-see-the-structure-of-a-table-in-sqlite
import sqlite3
connection = sqlite3.connect('shipment_database.db') # creates a connection to the on-disk database
cursor = connection.cursor() # creates database cursor, which enables us to execute SQL statements and fetch results from SQL queries

In [2]:
res = cursor.execute("SELECT name FROM sqlite_master")
res.fetchall() # shows all table names

[('product',), ('sqlite_autoindex_product_1',), ('shipment',)]

In [3]:
res = cursor.execute("SELECT sql FROM sqlite_schema WHERE name='shipment'")
res.fetchall() # shows relevant details of shipment table

[('CREATE TABLE shipment\n(\n    id          INTEGER not null primary key,\n    product_id  INTEGER not null references product,\n    quantity    INTEGER not null,\n    origin      TEXT    not null,\n    destination TEXT    not null\n)',)]

In [4]:
res = cursor.execute("SELECT sql FROM sqlite_schema WHERE name='product'")
res.fetchall() # shows relevant details of product table

[('CREATE TABLE product\n(\n    id   INTEGER not null primary key,\n    name TEXT    not null unique\n)',)]

#### Product     
id   INTEGER not null primary key     
name TEXT    not null unique

#### Shipment   
id          INTEGER not null primary key  
product_id  INTEGER not null references product  
quantity    INTEGER not null  
origin      TEXT    not null   
destination TEXT    not null

Useful docs
- https://docs.python.org/3/library/csv.html
- https://www.geeksforgeeks.org/reading-csv-files-in-python/

- Reads each row from the spreadsheets.
- Extracts the relevant data.
- Munges it into a format that fits the database schema.
- Inserts the data into the database.

#### Spreadsheet 0 

In [8]:
import csv
with open('data/shipping_data_0.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    id = 1
    for row in reader:
        # Insert into 'product' table
        cursor.execute('INSERT OR IGNORE INTO product (id, name) VALUES (?, ?)', (id, row[2]))
        # Check if insert actually happened
        if cursor.rowcount == 1:  # means a row was inserted
            id += 1
        

In [11]:
connection.commit() # save changes

In [12]:
# Check table insertions with
res = cursor.execute("SELECT * FROM product")
res.fetchall()

[(1, 'product'),
 (2, 'lotion'),
 (3, 'windows'),
 (4, 'skis'),
 (5, 'bikes'),
 (6, 'candy'),
 (7, 'capes'),
 (8, 'carnivorous plants'),
 (9, 'shoes'),
 (10, 'snakes'),
 (11, 'alternators'),
 (12, 'motor oil'),
 (13, 'trampolines'),
 (14, 'paint'),
 (15, 'hydroponic equipment'),
 (16, 'lamps'),
 (17, 'fruit'),
 (18, 'balloons'),
 (19, 'pants'),
 (20, 'apples'),
 (21, 'water bottles'),
 (22, 'pencil sharpeners'),
 (23, 'yarn'),
 (24, 'journals'),
 (25, 'socks'),
 (26, 'toys'),
 (27, 'coffee'),
 (28, 'board games'),
 (29, 'eggs'),
 (30, 'calculators'),
 (31, 'guitars'),
 (32, 'yoga mats'),
 (33, 'hvac'),
 (34, 'pencils'),
 (35, 'keyboards'),
 (36, 'earplugs'),
 (37, 'candles'),
 (38, 'fishing poles'),
 (39, 'hats'),
 (40, 'animal masks'),
 (41, 'furniture'),
 (42, 'baby food'),
 (43, 'microwaves'),
 (44, 'textbooks')]

In [13]:
# Must do this second because creation of product_id's is needed before inserting into shipment table
with open('data/shipping_data_0.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    id = 1
    for row in reader:
        res = cursor.execute('SELECT id FROM product WHERE name=?', (row[2],))
        product_id = res.fetchall()[0][0]
        # insert into 'shipment' table
        cursor.execute('INSERT INTO shipment (id, product_id, quantity, origin, destination) VALUES (?, ?, ?, ?, ?)', (id, product_id, row[4], row[0], row[1]))
        id += 1
        

In [14]:
# Check table insertions
res = cursor.execute('SELECT * FROM shipment')
res.fetchall()

[(1, 1, 'product_quantity', 'origin_warehouse', 'destination_store'),
 (2,
  2,
  59,
  'd5566b15-b071-4acf-8e8e-c98433083b2d',
  '50d33715-4c77-4dd9-8b9d-ff1ca372a2a2'),
 (3,
  3,
  28,
  'c42f0de8-b4f0-4167-abd1-ae79e5e18eea',
  '172eb8f3-1033-4fb6-b66b-d0df09df3161'),
 (4,
  4,
  63,
  'b145f396-de9b-42f1-9cc9-f5b52c3a941c',
  '65e4544d-42ae-4751-9580-bdcb90e5fcda'),
 (5,
  5,
  47,
  'f4372224-759f-43b3-bc83-ca6106bba1af',
  '745bee4e-710c-4538-8df1-5c146e1092a6'),
 (6,
  6,
  73,
  '49d0edae-9091-41bb-a08d-ab1c66bd08d5',
  '425b7a1a-b744-4c6b-898e-d424dd8cf18e'),
 (7,
  7,
  35,
  'b26d55e2-eb75-4bce-89cf-5cee095eaeea',
  '0495233b-9470-4e1a-bb30-a1ee3150c092'),
 (8,
  8,
  5,
  '6ae04f04-85df-4b89-b057-5cb0459baab7',
  'b0fcee8c-9047-405c-8316-28a38292be9d'),
 (9,
  9,
  85,
  '9a6ee454-f8ca-4a5a-b751-402ce8257b5b',
  '5958bf67-e190-4171-91eb-298b69c89282'),
 (10,
  10,
  34,
  '84003744-296e-4294-86a3-8a96439d4ee8',
  'd15bcc56-8948-4c3d-98ae-925d64b3f539'),
 (11,
  11,
  88,
  

In [15]:
connection.commit()

#### Spreadsheets 1 & 2

In [21]:
# If there are any new products in spreadsheet 1, add to the product table
with open('data/shipping_data_1.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        # in order to add new products, obtain the last product id and go from there
        res = cursor.execute('SELECT * FROM product ORDER BY id DESC LIMIT 1')
        last_index = res.fetchone()[0]
        cursor.execute('INSERT OR IGNORE INTO product (id, name) VALUES (?, ?)', (last_index+1, row[1]))

In [22]:
res = cursor.execute('SELECT * FROM product')
res.fetchall()

[(1, 'product'),
 (2, 'lotion'),
 (3, 'windows'),
 (4, 'skis'),
 (5, 'bikes'),
 (6, 'candy'),
 (7, 'capes'),
 (8, 'carnivorous plants'),
 (9, 'shoes'),
 (10, 'snakes'),
 (11, 'alternators'),
 (12, 'motor oil'),
 (13, 'trampolines'),
 (14, 'paint'),
 (15, 'hydroponic equipment'),
 (16, 'lamps'),
 (17, 'fruit'),
 (18, 'balloons'),
 (19, 'pants'),
 (20, 'apples'),
 (21, 'water bottles'),
 (22, 'pencil sharpeners'),
 (23, 'yarn'),
 (24, 'journals'),
 (25, 'socks'),
 (26, 'toys'),
 (27, 'coffee'),
 (28, 'board games'),
 (29, 'eggs'),
 (30, 'calculators'),
 (31, 'guitars'),
 (32, 'yoga mats'),
 (33, 'hvac'),
 (34, 'pencils'),
 (35, 'keyboards'),
 (36, 'earplugs'),
 (37, 'candles'),
 (38, 'fishing poles'),
 (39, 'hats'),
 (40, 'animal masks'),
 (41, 'furniture'),
 (42, 'baby food'),
 (43, 'microwaves'),
 (44, 'textbooks'),
 (45, 'keys'),
 (46, 'incense')]

In [23]:
connection.commit() # save changes

In [36]:
# Combine rows in spreadsheet 1 with same shipment identifier and product, write to new csv file
from collections import defaultdict

# Create a dictionary with key being the row and value being count
row_counts_dict = defaultdict(int)

with open('data/shipping_data_1.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        row_as_tuple = tuple(row) # Convert list to tuple so it can be used as a dict key
        row_counts_dict[row_as_tuple] += 1

with open ('shipping_data_1_combined.csv', 'w', newline='') as new_csvfile:
    writer = csv.writer(new_csvfile)
    for row, count in row_counts_dict.items():
        writer.writerow(list(row) + [count])

In [47]:
# Finally insert new rows into shipment table
with open('shipping_data_1_combined.csv', newline='') as first_csvfile:
    first_reader = csv.reader(first_csvfile)
    for row_from_first_csvfile in first_reader:
        # in order to insert new rows, obtain last rows id
        res = cursor.execute('SELECT * FROM shipment ORDER BY id DESC')
        last_index = res.fetchall()[0][0]

        # use product name to obtain product_id from product table
        res = cursor.execute('SELECT id FROM product WHERE name=?', (row_from_first_csvfile[1],))
        product_id = res.fetchall()[0][0]

        # obtain origin and destination
        with open('data/shipping_data_2.csv', newline='') as sec_csvfile:
            sec_reader = csv.reader(sec_csvfile)
            for row_from_sec_csvfile in sec_reader:
                if row_from_first_csvfile[0] == row_from_sec_csvfile[0]: # compare shipment identifiers
                    origin = row_from_sec_csvfile[1]
                    destination = row_from_sec_csvfile[2]

        cursor.execute("INSERT INTO shipment VALUES (?, ?, ?, ?, ?)", (last_index+1, product_id, row_from_first_csvfile[3], origin, destination))
        last_index += 1

In [49]:
res = cursor.execute('SELECT * FROM shipment')
res.fetchall()

[(1, 1, 'product_quantity', 'origin_warehouse', 'destination_store'),
 (2,
  2,
  59,
  'd5566b15-b071-4acf-8e8e-c98433083b2d',
  '50d33715-4c77-4dd9-8b9d-ff1ca372a2a2'),
 (3,
  3,
  28,
  'c42f0de8-b4f0-4167-abd1-ae79e5e18eea',
  '172eb8f3-1033-4fb6-b66b-d0df09df3161'),
 (4,
  4,
  63,
  'b145f396-de9b-42f1-9cc9-f5b52c3a941c',
  '65e4544d-42ae-4751-9580-bdcb90e5fcda'),
 (5,
  5,
  47,
  'f4372224-759f-43b3-bc83-ca6106bba1af',
  '745bee4e-710c-4538-8df1-5c146e1092a6'),
 (6,
  6,
  73,
  '49d0edae-9091-41bb-a08d-ab1c66bd08d5',
  '425b7a1a-b744-4c6b-898e-d424dd8cf18e'),
 (7,
  7,
  35,
  'b26d55e2-eb75-4bce-89cf-5cee095eaeea',
  '0495233b-9470-4e1a-bb30-a1ee3150c092'),
 (8,
  8,
  5,
  '6ae04f04-85df-4b89-b057-5cb0459baab7',
  'b0fcee8c-9047-405c-8316-28a38292be9d'),
 (9,
  9,
  85,
  '9a6ee454-f8ca-4a5a-b751-402ce8257b5b',
  '5958bf67-e190-4171-91eb-298b69c89282'),
 (10,
  10,
  34,
  '84003744-296e-4294-86a3-8a96439d4ee8',
  'd15bcc56-8948-4c3d-98ae-925d64b3f539'),
 (11,
  11,
  88,
  

In [50]:
connection.commit()